In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install ultralytics
!pip install deep_sort_realtime opencv-python-headless


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 110.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
print("Class names:", model.names)


Class names: {0: 'ball', 1: 'goalkeeper', 2: 'player', 3: 'referee'}


In [ ]:

import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

# Load YOLOv11 model
model_path = "/content/drive/MyDrive/best.pt"
model = YOLO(model_path)
print("✅ YOLOv11 model loaded.")

# Load DeepSORT
tracker = DeepSort(max_age=10)
print("✅ DeepSORT initialized.")

# Open input video
video_path = "/content/drive/MyDrive/15sec_input_720p.mp4"
cap = cv2.VideoCapture(video_path)

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter("deepsort_output.avi", cv2.VideoWriter_fourcc(*'XVID'), fps, (width, height))

# Frame-by-frame tracking
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLOv11
    results = model(frame)[0]

    detections = []
    for det in results.boxes:
        cls = int(det.cls)
        conf = float(det.conf)
        class_name = model.names[cls]
        if class_name == 'player':
            x1, y1, x2, y2 = map(int, det.xyxy[0])
            bbox = [x1, y1, x2 - x1, y2 - y1]  # Format: [x, y, w, h]
            detections.append((bbox, conf, class_name))

    # Run DeepSORT
    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        x1, y1, x2, y2 = map(int, track.to_ltrb())
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"ID {track_id}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    out.write(frame)

cap.release()
out.release()
print("✅ Video saved as deepsort_output.avi")


✅ YOLOv11 model loaded.
✅ DeepSORT initialized.

0: 384x640 1 ball, 16 players, 2 referees, 68.6ms
Speed: 2.3ms preprocess, 68.6ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 players, 2 referees, 40.5ms
Speed: 2.3ms preprocess, 40.5ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 16 players, 2 referees, 39.7ms
Speed: 2.3ms preprocess, 39.7ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 2 referees, 34.7ms
Speed: 2.3ms preprocess, 34.7ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 2 referees, 33.6ms
Speed: 2.4ms preprocess, 33.6ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 16 players, 2 referees, 33.2ms
Speed: 2.3ms preprocess, 33.2ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 players, 2 referees, 32.7ms
Speed: 2.3ms preprocess, 32.7

In [ ]:
from google.colab import files
files.download("deepsort_output.avi")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>